<h1 align="center">Prompt Engineering (Engenharia de Prompt)</h1>

Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro

Este script está baseado nos tutoriais de [James Briggs](https://www.youtube.com/@jamesbriggs/playlists).

Aqui, exploraremos os fundamentos da `Engenharia de Prompt`. Começaremos instalando a biblioteca `openai`, que usaremos ao longo desses exemplos. No entanto, observe que podemos usar outros `LLMs` aqui, como os oferecidos pelo Cohere ou alternativas de código aberto disponíveis via [Hugging Face](https://huggingface.co/).

In [ ]:
%pip install openai

# Estrutura de um Prompt

Um prompt pode consistir em vários componentes:

* Instruções

* Informação externa ou contexto

* Entrada ou consulta (`query`) do usuário

* Indicador de saída

Nem todos os prompts requerem todos esses componentes, mas geralmente um bom prompt usará dois ou mais deles. Vamos definir o que todos eles são com mais precisão.

As <font color="red">Instruções</font> dizem ao modelo o que fazer, normalmente como ele deve usar entradas e/ou informações externas para produzir a saída que queremos.

<font color="red">Informações externas ou contexto</font> são informações adicionais que inserimos manualmente no prompt, recuperamos por meio de um banco de dados vetorial (memória de longo prazo) ou extraímos por outros meios (chamadas de `API`, cálculos etc.).

A <font color="red">Entrada ou consulta do usuário</font> é normalmente uma consulta inserida diretamente pelo usuário do sistema.

O <font color="red">Indicador de saída</font> é o início do texto gerado. Para um modelo que gera código Python, podemos colocar `import` (já que a maioria dos scripts Python começa com uma biblioteca `import`) ou um `chatbot` pode começar com `Chatbot`:(supondo que formatemos o script do chatbot como linhas de texto intercambiável entre `User` e `Chatbot`).



Cada um desses componentes geralmente deve ser colocado na ordem em que os descrevemos. <font color="orange">Começamos com instruções, fornecemos contexto (se necessário), depois adicionamos a entrada do usuário e, finalmente, terminamos com o indicador de saída.</font>